In [1]:
# !pip install tweepy

In [2]:
import pandas as pd
import numpy as np
import json
import tweepy

In [3]:
import config

In [4]:
# df_ = pd.read_csv('all_labeled_data_without_maryland.csv')
# df_.shape

In [5]:
# df_.head()

In [6]:
# get the list of indices to retrieve
# ids = list(df_['tweet_id'])

In [7]:
client = tweepy.Client( 
                bearer_token = config.Bearer_Token,
                consumer_key = config.API_Key,
                consumer_secret = config.API_Secret,
                wait_on_rate_limit=True
         )

In [8]:
# conversation id is the original tweet, if this is the same or different from the tweet id could be significant

tweet_fields = ["author_id", 
                "created_at", 
                "conversation_id", 
                "context_annotations",
                "edit_history_tweet_ids",
                "attachments",
                "entities",
                "in_reply_to_user_id",
                 "possibly_sensitive",
                 "public_metrics",
                 "lang",
                 "referenced_tweets",
                 "reply_settings",
                 "source"
                 ]

In [9]:
def retrieve_tweets(index_list):
    list_of_tweets = []
    try:
        tweets = client.get_tweets(ids = index_list, tweet_fields = tweet_fields)
        for tweet in tweets.data:
            current_tweet = {
                'tweet_id': tweet.id,
                'text': tweet.text,
                'author_id': tweet.author_id,
                'created_at': tweet.created_at or np.nan,
                'conversation_id': tweet.conversation_id or np.nan,
                'entities': tweet.entities or np.nan,
                'context_annotations': tweet.context_annotations or np.nan,
                "edit_history_tweet_ids": tweet.edit_history_tweet_ids or np.nan,
                "in_reply_to_user_id": tweet.in_reply_to_user_id or np.nan,
                "attachments": tweet.attachments or np.nan,
                "lang": tweet.lang or np.nan, 
                "possibly_sensitive": tweet.possibly_sensitive or np.nan,
                "public_metrics": tweet.public_metrics or np.nan,
                "referenced_tweets": tweet.referenced_tweets or np.nan,
                "reply_settings": tweet.reply_settings or np.nan,
                "source": tweet.source or np.nan 
            }
            current_df = pd.DataFrame([current_tweet])
            list_of_tweets.append(current_df)
        df = pd.concat(list_of_tweets)
        return df

    except Error as e:
        print("Error:", e)
        return False

In [10]:
def collect_tweet_dfs(index_list, start_twt):
    
    end_twt = start_twt + 100

    tweet_dfs = []

    working = True

    while working: 
        try:
            batch = index_list[start_twt: end_twt]
            
            df =  retrieve_tweets(batch)
            tweet_dfs.append(df)
            start_twt += 100
            end_twt += 100

        except Exception as e:
            working = False
            print(end_twt)
            print(e)
                 
    
    total_dfs = pd.concat(tweet_dfs)
    return total_dfs

In [11]:
short_list = [ 910783391829598208,
 913032371435433984,
 914175720968028161,
 910549451214872577,
 914176943033024514,
 913879154789056512,
 914524833047810048,
 912385296557334530,
 912218742930550784,
 912123826217025536]

In [12]:
%%time
df = collect_tweet_dfs(short_list, 0)

200
name 'Error' is not defined
CPU times: user 55.1 ms, sys: 3.24 ms, total: 58.3 ms
Wall time: 507 ms


In [13]:
df = df.fillna("no value")

In [14]:
df.head()

,tweet_id,text,author_id,created_at,conversation_id,entities,context_annotations,edit_history_tweet_ids,in_reply_to_user_id,attachments,lang,possibly_sensitive,public_metrics,referenced_tweets,reply_settings,source
0,914175720968028161,@alan5422 @nytimes U.S. boots have been on the...,241033134,2017-09-30 17:10:51+00:00,914109786341429248,"{'urls': [{'start': 119, 'end': 142, 'url': 'h...","[{'domain': {'id': '47', 'name': 'Brand', 'des...",[914175720968028161],588769770.0,{'media_keys': ['3_914175714278105088']},en,no value,"{'retweet_count': 0, 'reply_count': 1, 'like_c...","[(type, id), (type, id)]",everyone,Twitter for iPhone
0,910549451214872577,Hurricane Maria is ravaging the same islands I...,463325432,2017-09-20 17:01:21+00:00,910549451214872577,"{'urls': [{'start': 91, 'end': 114, 'url': 'ht...","[{'domain': {'id': '29', 'name': 'Events [Enti...",[910549451214872577],no value,no value,en,no value,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",no value,everyone,Twitter Web Client
0,913879154789056512,RT @JRodzMIA: #SanLorenzo #PuertoRico left in ...,798303441802186752,2017-09-29 21:32:24+00:00,913879154789056512,"{'annotations': [{'start': 15, 'end': 24, 'pro...",no value,[913879154789056512],no value,no value,en,no value,"{'retweet_count': 1, 'reply_count': 0, 'like_c...","[(type, id)]",everyone,Twitter for iPhone
0,912385296557334530,RT @Dixiepro: Is our Government really going f...,101938339,2017-09-25 18:36:20+00:00,912385296557334530,"{'annotations': [{'start': 69, 'end': 70, 'pro...","[{'domain': {'id': '29', 'name': 'Events [Enti...",[912385296557334530],no value,no value,en,no value,"{'retweet_count': 1, 'reply_count': 0, 'like_c...","[(type, id)]",everyone,Twitter for iPad
0,912218742930550784,RT @USDAFoodSafety: Currently affected by a po...,59714064,2017-09-25 07:34:31+00:00,912218742930550784,"{'annotations': [{'start': 132, 'end': 138, 'p...",no value,[912218742930550784],no value,no value,en,no value,"{'retweet_count': 49, 'reply_count': 0, 'like_...","[(type, id)]",everyone,Twitter Web Client


In [15]:
df.shape

(6, 16)

In [16]:
df_ = df_[['tweet_id', 'class_label']]

NameError: name 'df_' is not defined

In [ ]:
df_full = df_.merge(df, on='tweet_id', how='outer')

In [ ]:
df_full.shape

In [ ]:
# df_full = df_full.dropna()
# df_full.shape

In [ ]:
df_full.head()

In [ ]:
27443 - 18787

In [ ]:
df_full.shape

In [ ]:
df_created = df_full[['tweet_id', 'class_label', 'text', 'created_at']]

In [ ]:
df_created.shape

In [ ]:
# df_full.to_csv('labeled_with_features.csv', index=False)

In [ ]:
df_3 = df_created[df_created['created_at'] == datetime.isnat()]
df_3.shape

In [ ]:
list(df_created['created_at'].unique())

In [ ]:
times = df_3['created_at'].astype('str')

In [ ]:
times

In [ ]:
# d = "BAM!  " * 1000
# print(d)